## 통계검정
- 교재 p.294

## 문제 상황
- 편의점 감자튀김 (무게 130g으로 알려져 있음)
- 학생이 하나 사서 무게를 재보니 122.02g 밖에 되지 않음
- 2주간 매일감자튀김을 사서 무게를 측정
    + 14개의 표본의 평균 계산했는데, 128.451g  => 학생이 편의점주에게 항의
    + 편의점주: 그저 우연이다!
- 14개 표본의 평균이 128.451g으로 나온건 정말 우연인가?

### 질문
- 어떤 통계 검정 방법을 사용해야 할까요?
    + t-test, one sample T-test
    + 표본 14개 ==> 정규성 검정 진행 후, 정규성 검정 통과할 때, One sample T-test
- 귀무가설과 대립가설을 설정해보세요
    + 귀무가설: 모평균(130g)과 표본평균 (128.451g)과 같다.
    + 대립가설: 모평균(130g)과 표본평균 (128.451g)과 같지 않다.

In [2]:
import numpy as np
import pandas as pd
from scipy import stats

In [26]:
%precision 3
np.random.seed(1111)

In [27]:
df = pd.read_csv('data/ch11_potato.csv')
df

,무게
0,122.02
1,131.73
2,130.60
3,131.82
4,132.05
5,126.12
6,124.43
7,132.89
8,122.79
9,129.95


In [28]:
sample = np.array(df['무게'])
sample

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [29]:
# 표본평균
s_mean = np.mean(sample)
s_mean

128.451

### A 학생의 관심
- (14개의) 표본평균이 모평균 130g보다 적은지 여부
- 여기서는 감자튀김의 표본집단이 정규분포를 따르고 있고 ==> 정규성 검증을 할 필요가 있을까요?
    + 정규성 검증을 할 필요가 없다: 정규분포를 따르고 있기 때문에
    + 비모수 검정을 쓸 필요가 없다!!!
    + 모수 검정을 하면 된다!!
- 모분산이 9임을 알고 있다고 전제한다.

In [30]:
rv = stats.norm(130, #모평균
               np.sqrt(9/14)) # 모분산, 14 표본의 갯수
rv.isf(0.95)  # 0.95 신뢰구간 95%

128.681

In [31]:
rv = stats.norm(130, #모평균
               np.sqrt(9/30)) # 모분산, 14 표본의 갯수
rv.isf(0.95)  # 0.95 신뢰구간 95%

129.099

#### 검정통계량

In [32]:
# 모평균 130
z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932

In [33]:
# 검정 통계량에 관한 임곗값을 구하기
rv = stats.norm()
rv.isf(0.95)

-1.645

In [34]:
# 모평균 130
z = (s_mean - 130) / np.sqrt(9/14)
rv = stats.norm()
rv.isf(0.95)

-1.645

In [35]:
rv.cdf(z)  # 유의수준 0.05보다 작음

# 귀무가설: 모평균 130g 표본평균 128.451g과 같다
# 대립가설: 모평균 140g 표본평균 128.451g과 다르다

# 대립가설 채택을 한다는 말은 통계에서는 존재하지 않음
# 귀무가설 채택 or 귀무가설 기각(= 대립가설 채택)

0.027

In [36]:
# 양측검정
z = (s_mean - 130) / np.sqrt(9/14)
rv = stats.norm()
rv.interval(0.95)  # 신뢰구간 95%

(-1.960, 1.960)

In [37]:
rv.cdf(z) * 2

0.053

## 가설검정의 두 가지 오류
- 제 1종 오류: 귀무가설이 옳을 때 귀무가설을 기각하는 오류
- 제 2종 오류: 대립가설이 옳을 때 귀무가설을 채택하는 오류

### 제 1종 오류
- 실제로 평균이 130g인데도 불구하고 평균은 130g보다 작다라고 결론을 내리는 상황
    + False Positive(오탐)

In [39]:
rv = stats.norm(130, 3)
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z < c:
        cnt += 1
cnt / n_samples

0.052

- 1종 오류를 범할 확률은 0.053 / 약 5%의 비율로 130g보다 작다라고 잘못 탐지하는 것과 같다.
- 1종 오류를 범할 확률을 위험률
- 좀 더 엄격하게 적용하고 싶다고 하면 유의수준 1%에서 가설검정을 수행

### 제 2종 오류
- 실제로 표본평균이 130g보다 작음에도 불구하고 표본평균은 130g보다 작다라는 결론을 얻을 수 없는 상황
    + 본래 검출해야 하는 것을 검출하지 못했으므로 미탐 false negative

In [42]:
rv = stats.norm(128, 3)  # A학생이 편의점에서 비밀 문서를 확인, 실제로는 감자튀김의 평균이 128g
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z >= c:
        cnt += 1
        
cnt / n_samples

0.196

- 0.198의 의미는 검정력 / 제대로 탐지못할 확률이 20%나 되더라!
- 제 2종 오류는 언제나 모집단의 정보에 의존한다.
    + 중요한 포인트는 분석가는 모집단의 구체적인 정보를 알 방법이 없다.

### 모분산을 안다!!!
- 현실적으로 모집단을 아는 것은 불가능
    + 건강보험과 통계 / 전국민 데이터가 있는 자료 (국가)
- 모분산을 알고 있다는 뜻은 모집단을 알고 있다  => 모평균도 구할 수 있음 ==> 모평균에 대한 검정을 우리가 굳이 할 필요가 있는가?

In [52]:
def pmean_test(sample, mean0, alpha=0.05):
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1-alpha)

    t = (s_mean - mean0) / np.sqrt(u_var/n)
    print(f't통계량 값: {t:.3f}')
    if interval[0] <= t <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')

    if t < 0:
        p = rv.cdf(t) * 2
    else:
        p = (1 - rv.cdf(t)) * 2
    print(f'p값은 {p:.3f}')

In [53]:
pmean_test(sample, 130)

t통계량 값: -1.455
귀무가설을 채택
p값은 0.169


In [51]:
t, p = stats.ttest_1samp(sample, 130)
t, p

(-1.455, 0.169)

### z 통계량 / t 통계량
- 모집단을 알고, 모평균을 알고, 모분산을 다 알고 있다.
    + z-검정 : z-통계량
    
- 표본을 추출한다. 쵸본을 알고, 표본 분산을 알고 있다.
    + t-검정 : t-통계량

# 2표본 문제에 관한 가설검정
- 대응표본: 정규분포를 가정할 수 있음
    + 정규성 검정 통과: 대응표본 t-검정
    + 정규성 검정 통과 실패: 비모수 검정, 윌콕슨으 부호순위검정

- 독립표본: 정규분포를 가정할 수 있음
    + 정규성 검정 통과: 독립성 비교 t-검정
    + 정규성 검정 통과 실패: 맨-위트니의 U 검정
    
- 기준 샘플 30개
    + 석사 수준에서는 교수님들이 한마디 던짐 ==> 샘플이 너무 적다!

### 대응표본 t-검정
- A 학생은 친구 20명에게 1주일간 근력 운동을 하게 함 (Intervention, 개입)
- 운동 전후에 집중력을 측정하는 테스트 받게 됨
- 대응표본 t-검정: 같은 집단에 대한 전후 비교를 할 때 대응표본 t-검정을 사용하기 때문입니다.
- 가설검정
    + 귀무가설: 운동 전후 집중력에 차이가 없다.
    + 대립가설: 운동 전후 집중력에 차이가 있다.

In [56]:
import pandas as pd

training_rel = pd.read_csv('data/ch11_training_rel.csv')
training_rel['차'] = training_rel['후'] - training_rel['전']
training_rel

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25
5,45,37,-8
6,55,60,5
7,49,52,3
8,49,68,19
9,52,70,18


In [58]:
t, p = stats.ttest_1samp(training_rel['차'], 0)
t, p 

(2.204, 0.040)

- 보고서 작성
    + t-통계량 / p-value 값

## 독립비교 t-검정
- 교재는 p318
- 학생이 부쩍 늘었음
    + A 학생 그룹 인문계열
    + B 학생 그룹 체육계열
- A 그룹, B 그룹 사이에 집중력 테스트의 평균에서 차이가 나지 않을까 생각하여 테스트를 함
- 이 데이터로부터 어떤 검정을 수행해야 A 학생과 B 학생 학급의 집중력에 유의한 차이가 있는지 확인
    + 귀무가설: A 학생 그룹과 B 학생 그룹의 집중력 테스트 평균에는 차이가 없다.
    + 대립가설: A 학생 그룹과 B 학생 그룹의 집중력 테스트 평균에는 차이가 있다.
- 독립비교 t-검정 두 가지 검정
    + 1) 각 그룹 데이터의 정규성을 가짐
    + 2) 두 그룹의 분산이 같다 (=등분산성)

In [59]:
training_ind = pd.read_csv('data/ch11_training_ind.csv')
training_ind.head() # training_ind.shape

,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


In [61]:
# p320
t, p = stats.ttest_ind(training_ind['A'], training_ind['B'], equal_var = True)  # 등분산성이 같다!
t, p

(-1.761, 0.086)

- 결론: 두 그룹으 평균 차이는 없더라, 귀무가설 채택 

In [62]:
# p320
t, p = stats.ttest_ind(training_ind['A'], training_ind['B'], equal_var = False)  # 등분산성이 같다!
t, p

(-1.761, 0.087)

## 비모수 검정
- 모수 검정에서 각 가정을 만족시키지 못할 때, 비모수검정을 쓰더라
### 윌콕슨으 부호순위검정
- 대응표본에서 차이에 정규분포를 만족시키지 못할때
    + 평균차이 아님 (X)
    + 중앙값 차이에 대한 검정 (O)
    + 중앙값 = 50%에 해당하는 값을 

In [65]:
training_rel = pd.read_csv('data/ch11_training_rel.csv')
toy_df = training_rel[:6].copy()
toy_df['차'] = toy_df['후']- toy_df['전']
toy_df

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25
5,45,37,-8


In [67]:
# 순위 적용
diff = toy_df['후']- toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,41,-18,5
1,52,63,11,3
2,55,68,13,4
3,61,59,-2,1
4,59,84,25,6
5,45,37,-8,2


In [69]:
import numpy as np

r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus  # 검정통계량 8

(8, 13)

In [74]:
toy_df['후'] = toy_df['전'] + np.arange(1, 7)
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['차'] = diff
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,60,1,1
1,52,54,2,2
2,55,58,3,3
3,61,65,4,4
4,59,64,5,5
5,45,51,6,6


In [73]:
toy_df['후'] = toy_df['전'] + np.arange(1, 7)
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['차'] = diff
toy_df['순위'] = rank
toy_df

r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus  # 검정통계량 0

(0, 21)

In [75]:
toy_df['후'] = toy_df['전'] + [1, -2, -3, 4, 5, -6]
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['차'] = diff
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,60,1,1
1,52,50,-2,2
2,55,52,-3,3
3,61,65,4,4
4,59,64,5,5
5,45,39,-6,6


In [77]:
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus  # 검정통계량 10

(11, 10)